In [1]:
import tika
import tika.parser
from tika import parser
import requests
import urllib.parse
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#run "python -mhttp.server 8888" in the directory where your photos are

#run ifconfig in terminal to find your ip address

images_url_usc  = 'http://10.25.179.208:8888'

images_url_home = 'http://192.168.1.13:8888'

images_url_MOC = 'http://10.25.188.76:8888'

# Webscraping to get list of image URLs

In [4]:
# Creating a list of urls from webpage 
# This works!


webpage_url = images_url_home

response = requests.get(webpage_url)
html_content = response.content

soup = BeautifulSoup(html_content, 'html.parser')

image_urls = []
for i, link in enumerate(soup.find_all('a')):
    if i >= 20:
        break
    href = link.get('href')
    if href is not None and href.startswith('http'):
        image_urls.append(href)
    elif href is not None:
        image_urls.append(webpage_url + '/' + href)

#print(image_urls)


# Image Captions DF

- High level strategy: create a list of key value pairs, and save it to a JSON file periodically so that if the program crashes I don't lose my progress


In [5]:
#docker run -it -p 8764:8764 uscdatascience/im2txt-rest-tika
#test_url = 'http://localhost:8764/inception/v3/caption/image?url=http://192.168.1.13:8888/Pixstory-image-164318304274781.png'

captions_results = []

# Loop through the image URLs and classify each image
for url in image_urls:

    # Construct the API URL for the image classification
    api_captions_url = 'http://localhost:8764/inception/v3/caption/image?url=' + url

    # Send the API request
    response = requests.get(api_captions_url)

    # Parse the JSON response
    json_response = response.json()

    # Extract the first listed sentence
    first_sentence = json_response['captions'][0]['sentence']

    # Add the results to the list
    captions_results.append({

        'url': url, 
        'caption':first_sentence
    })

# Create a Pandas dataframe from the list of dictionaries
captions_df = pd.DataFrame(captions_results)

# Set display options to show all columns and avoid truncation of string data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Display the dataframe
captions_df

,url,caption
0,http://192.168.1.13:8888/Pixstory-image-161668402049563.jpg,a bunch of bananas that are on a table .
1,http://192.168.1.13:8888/Pixstory-image-161681318757326.jpg,a man sitting on a couch with a laptop .
2,http://192.168.1.13:8888/Pixstory-image-161984246212221.jpeg,a man in a suit and tie holding a cell phone .
3,http://192.168.1.13:8888/Pixstory-image-162014760674045.jpeg,a bunch of birds that are flying in the air .
4,http://192.168.1.13:8888/Pixstory-image-162051250471634.jpeg,a little girl sitting in a chair holding a teddy bear .
5,http://192.168.1.13:8888/Pixstory-image-162153257364412.png,a bike parked on the side of a road .
6,http://192.168.1.13:8888/Pixstory-image-162157341332029.jpeg,a man in a suit and tie standing in a room .
7,http://192.168.1.13:8888/Pixstory-image-162194441755927.jpeg,a red fire hydrant sitting in the middle of a forest .
8,http://192.168.1.13:8888/Pixstory-image-162244448366044.jpeg,a bunch of flowers that are on a table .
9,http://192.168.1.13:8888/Pixstory-image-16227252974324.png,a close up of a person holding a pair of scissors


# Image Objects DF

In [10]:
#docker run -it -p 8764:8764 uscdatascience/inception-rest-tika
# test_url_objects = 'http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=http://192.168.1.13:8888/Pixstory-image-164318304274781.png'

# List to store the classification results
objects_results = []

# Loop through the image URLs and classify each image
for url in image_urls:

    # Construct the API URL for the image classification
    api_objects_url = 'http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=' + url

    # Send the API request
    response = requests.get(api_objects_url)

    # Parse the JSON response
    json_response = response.json()

    # Extract the top two labels and their confidences from the JSON response
    classnames = json_response['classnames']
    confidence = json_response['confidence']


    # Add the results to the list
    objects_results.append({

        'url': url,
        'detect object': classnames,
    

    })

# Create a Pandas dataframe from the list of dictionaries
objects_df = pd.DataFrame(objects_results)


objects_df


,url,detect object
0,http://10.25.188.76:8888/Pixstory-image-161668...,"[coral fungus, hen-of-the-woods, hen of the wo..."
1,http://10.25.188.76:8888/Pixstory-image-161681...,"[studio couch, day bed]"
2,http://10.25.188.76:8888/Pixstory-image-161984...,"[Windsor tie, suit, suit of clothes]"
3,http://10.25.188.76:8888/Pixstory-image-162014...,"[tray, handkerchief, hankie, hanky, hankey]"
4,http://10.25.188.76:8888/Pixstory-image-162051...,"[sweatshirt, poncho]"
5,http://10.25.188.76:8888/Pixstory-image-162153...,"[oxcart, horse cart, horse-cart]"
6,http://10.25.188.76:8888/Pixstory-image-162157...,"[suit, suit of clothes, Windsor tie]"
7,http://10.25.188.76:8888/Pixstory-image-162194...,"[pot, flowerpot, picket fence, paling]"
8,http://10.25.188.76:8888/Pixstory-image-162244...,"[pot, flowerpot]"
9,http://10.25.188.76:8888/Pixstory-image-162272...,"[book jacket, dust cover, dust jacket, dust wr..."


# Merging objects and captions df

In [10]:
# merge the dataframes based on the 'url' column
captions_objects_df = pd.merge(captions_df, objects_df, on='url', how='outer')

captions_objects_df

,url,caption,detect object
0,http://192.168.1.13:8888/Pixstory-image-161668402049563.jpg,a bunch of bananas that are on a table .,"[coral fungus, hen-of-the-woods, hen of the woods, Polyporus frondosus, Grifola frondosa]"
1,http://192.168.1.13:8888/Pixstory-image-161681318757326.jpg,a man sitting on a couch with a laptop .,"[studio couch, day bed]"
2,http://192.168.1.13:8888/Pixstory-image-161984246212221.jpeg,a man in a suit and tie holding a cell phone .,"[Windsor tie, suit, suit of clothes]"
3,http://192.168.1.13:8888/Pixstory-image-162014760674045.jpeg,a bunch of birds that are flying in the air .,"[tray, handkerchief, hankie, hanky, hankey]"
4,http://192.168.1.13:8888/Pixstory-image-162153257364412.png,a bike parked on the side of a road .,"[oxcart, horse cart, horse-cart]"
5,http://192.168.1.13:8888/Pixstory-image-162157341332029.jpeg,a man in a suit and tie standing in a room .,"[suit, suit of clothes, Windsor tie]"
6,http://192.168.1.13:8888/Pixstory-image-162244448366044.jpeg,a bunch of flowers that are on a table .,"[pot, flowerpot]"
7,http://192.168.1.13:8888/Pixstory-image-16233178221386.jpeg,a man in a suit and tie standing in a room .,"[lab coat, laboratory coat]"
8,http://192.168.1.13:8888/Pixstory-image-162347005942405.jpeg,a close up of a vase with flowers in it,"[coral fungus, matchstick]"
9,http://192.168.1.13:8888/Pixstory-image-162454466467149.jpeg,a cat laying on top of a blanket on a bed .,"[tabby, tabby cat, tiger cat]"
